In [1]:
from pathlib import Path
import pandas as pd
from typing import List

In [2]:
base_path = Path("/workspace/DiffusionDrive/navsim_workspace/exp")
csv_paths = list(base_path.rglob("*.csv"))
csv_paths = [p for p in csv_paths if "eval" in str(p)]
csv_paths

[PosixPath('/workspace/DiffusionDrive/navsim_workspace/exp/eval_diffusiondrive_agent_default_bs256ep300_epoch=299-step=12600/2025.06.20.08.23.55/2025.06.20.08.36.15.csv'),
 PosixPath('/workspace/DiffusionDrive/navsim_workspace/exp/diffusiondrive_agent_original_public_eval/2025.06.18.06.48.59/2025.06.18.06.58.56.csv'),
 PosixPath('/workspace/DiffusionDrive/navsim_workspace/exp/eval_diffusiondrive_agent_default_bs32ep300_epoch=59-step=19980/2025.06.20.07.24.22/2025.06.20.07.34.39.csv'),
 PosixPath('/workspace/DiffusionDrive/navsim_workspace/exp/eval_diffusiondrive_agent_default_bs32ep300_epoch=49-step=16650/2025.06.20.07.34.51/2025.06.20.07.45.00.csv'),
 PosixPath('/workspace/DiffusionDrive/navsim_workspace/exp/eval_diffusiondrive_agent_default_bs256ep300_epoch=999-step=42000/2025.06.20.07.45.15/2025.06.20.07.55.18.csv'),
 PosixPath('/workspace/DiffusionDrive/navsim_workspace/exp/eval_diffusiondrive_agent_default_bs64ep300_epoch=9-step=1670/2025.06.20.05.55.19/2025.06.20.06.07.37.csv'),


In [3]:
def compare_result(paths: List[Path]):
    df_lst = []
    names_lst = []
    for path in paths:
        df_lst.append(pd.read_csv(path).iloc[-1:,3:])
        names_lst.append(path.parents[1].name)
    df = pd.concat(df_lst, keys=names_lst)
    df = df.reset_index(names=["model", "index"]).drop(columns=["index"])
    
    # Extract epoch from model name
    epoch_col = df["model"].str.extract(r'epoch=(\d+)')
    df.insert(1, "epoch", epoch_col)
    # The default value for epoch is 99
    df["epoch"] = df["epoch"].fillna(99)
    df["epoch"] = df["epoch"].astype(int)
    
    # Extract batch size from model name
    batch_size_col = df["model"].str.extract(r'bs(\d+)')
    df.insert(2, "batch_size", batch_size_col)
    # The default value for batch size is 64
    df["batch_size"] = df["batch_size"].fillna(64)
    df["batch_size"] = df["batch_size"].astype(int)
    
    df["model"] = df["model"].str.replace("eval_diffusiondrive_agent_default_", '')
    df["model"] = df["model"].str.replace("diffusiondrive_agent_", '')

    
    return df.style.highlight_max(color='purple', axis=0)
compare_result(csv_paths)

,model,epoch,batch_size,no_at_fault_collisions,drivable_area_compliance,ego_progress,time_to_collision_within_bound,comfort,driving_direction_compliance,score
0,bs256ep300_epoch=299-step=12600,299,256,0.978511,0.943685,0.803084,0.937510,0.999753,0.976288,0.858189
1,original_public_eval,99,64,0.982175,0.962374,0.822096,0.947472,0.999835,0.980899,0.880551
2,bs32ep300_epoch=59-step=19980,59,32,0.982299,0.958423,0.820187,0.944508,0.999918,0.981681,0.876496
3,bs32ep300_epoch=49-step=16650,49,32,0.982669,0.962786,0.824923,0.939322,1.000000,0.981681,0.878875
4,bs256ep300_epoch=999-step=42000,999,256,0.982340,0.950683,0.812336,0.942944,0.999918,0.976577,0.868817
5,bs64ep300_epoch=9-step=1670,9,64,0.974148,0.906636,0.762364,0.924831,0.997777,0.970649,0.814420
6,ep300_epoch=299-step=50100,299,64,0.981064,0.964844,0.823276,0.944426,0.999918,0.980487,0.881188
7,bs32ep300_epoch=199-step=66600,199,32,0.978141,0.960234,0.823759,0.931665,0.999753,0.979623,0.872957
8,default_eval,99,64,0.980199,0.961633,0.820798,0.941956,0.999918,0.981105,0.877446
9,bs64ep100_epoch=99-step=16700,99,64,0.980199,0.961304,0.820510,0.941709,0.999918,0.980652,0.877081
